# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [1]:
import pandas as pd
import random
import os
from helpers.paths import PathMerger
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [2]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'id': 1, 
        'username': 'janisourander@kamk.fi',
        # 'password': fake.md5(),
        'created': datetime(1970, 1, 15, 10, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
             # 'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 2, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 3, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 4, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    # i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'id': len(devices) + 1, 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(details) + 1,
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [3]:
# Table paths
table_paths = {}

# Map of combinations to create: (db, table, data_generator_function)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    dt = f'{db}.{table}'
    sp = PathMerger(db, table).staging
    dp = os.path.join(sp, 'LOAD00000001.parquet')
    
    # Create dirs
    if not os.path.exists(sp):
        os.makedirs(sp)
        
    table_paths[dt] = dp

## Generate Datasets

### Customers

In [4]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,dms_timestamp,id,username,created,modified
0,2021-08-15 09:40:46,1,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56
1,2021-08-15 09:40:46,1,tonywilliams@barnes.com,2020-01-15 13:37:00,2020-07-13 08:26:21
2,2021-08-15 09:40:46,2,cmarsh@yahoo.com,2020-01-15 13:39:56,2020-10-08 10:25:34
3,2021-08-15 09:40:46,3,carriegreen@marquez-gray.com,2020-01-15 14:00:09,2020-04-27 13:42:13
4,2021-08-15 09:40:46,4,hthomas@yahoo.com,2020-01-15 14:05:31,2020-08-06 01:27:09
5,2021-08-15 09:40:46,5,wbishop@hotmail.com,2020-01-15 14:50:35,2020-09-30 20:13:59
6,2021-08-15 09:40:46,6,steven77@martinez.com,2020-01-15 15:00:34,2020-04-10 23:27:16
7,2021-08-15 09:40:46,7,smithjames@gmail.com,2020-01-15 15:01:20,2020-12-13 10:11:28
8,2021-08-15 09:40:46,8,gperez@hotmail.com,2020-01-15 15:24:21,2020-05-09 21:45:26
9,2021-08-15 09:40:46,9,khale@hotmail.com,2020-01-15 16:04:34,2020-11-29 22:48:03


### Device Models

In [5]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,dms_timestamp,id,release_date,name,color,description,created,modified
0,2021-08-15 09:40:46,1,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,2021-08-15 09:40:46,2,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2021-08-15 09:40:46,3,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,2021-08-15 09:40:46,4,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [6]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_d.to_parquet(table_paths['devices.devices'])

# Show
df_d

,dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
0,2021-08-15 09:40:46,1,1,2,846-09-7018,1970-01-15 10:40:34,1970-01-15 11:19:42
1,2021-08-15 09:40:46,2,1,4,814-78-8514,1970-01-15 10:40:34,1970-01-15 11:25:19
2,2021-08-15 09:40:46,3,1,4,305-68-2901,2020-01-15 14:02:05,2020-01-15 14:33:37
3,2021-08-15 09:40:46,4,1,1,064-41-8841,2020-01-15 14:02:05,2020-01-15 14:28:25
4,2021-08-15 09:40:46,5,2,4,755-25-7677,2020-01-15 13:40:13,2020-01-15 13:48:17
5,2021-08-15 09:40:46,6,2,2,827-85-3026,2020-01-15 13:40:13,2020-01-15 13:59:16
6,2021-08-15 09:40:46,7,3,4,838-29-0909,2020-01-15 14:02:51,2020-01-15 14:50:53
7,2021-08-15 09:40:46,8,3,3,090-39-2995,2020-01-15 14:02:51,2020-01-15 15:02:30
8,2021-08-15 09:40:46,9,4,2,388-11-5076,2020-01-15 14:29:49,2020-01-15 14:34:36
9,2021-08-15 09:40:46,10,5,2,268-23-3325,2020-01-15 15:06:57,2020-01-15 16:05:29


### Customer_details

In [7]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_cd.to_parquet(table_paths['customers.customer_details'])

df_cd

,dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,2021-08-15 09:40:46,1,1,1942-09-01,gez,19254 Le Shoal,96842,West Jessicafurt,BG,001-732-790-0908,1970-01-15 10:00:00,1970-01-15 10:00:00
1,2021-08-15 09:40:46,2,1,1988-09-30,lo,709 Dawn Crescent,47467,Stewartview,KR,+1-702-616-0085,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2021-08-15 09:40:46,3,2,1960-07-28,en,93565 Jessica Fields Suite 008,97313,Porterport,CN,858.030.0424x156,2020-01-15 13:39:56,2020-01-15 13:39:56
3,2021-08-15 09:40:46,4,3,1921-01-21,ur,11739 Hernandez Orchard Suite 495,05875,Michaelfurt,HU,001-421-905-1192x1044,2020-01-15 14:00:09,2020-01-15 14:00:09
4,2021-08-15 09:40:46,5,4,1910-07-21,hr,182 Terry View,67252,Parkborough,AU,999.657.5608,2020-01-15 14:05:31,2020-01-15 14:05:31
5,2021-08-15 09:40:46,6,5,2019-08-29,mag,369 Smith Forks Suite 570,60978,New Jesse,GY,395-576-0375x91476,2020-01-15 14:50:35,2020-01-15 14:50:35
6,2021-08-15 09:40:46,7,6,1929-12-16,tg,600 Marvin Rue,00647,New Ambershire,LT,2269049584,2020-01-15 15:00:34,2020-01-15 15:00:34
7,2021-08-15 09:40:46,8,7,2013-05-10,kk,48189 Jessica Alley,25141,North Chadton,MK,5446683628,2020-01-15 15:01:20,2020-01-15 15:01:20
8,2021-08-15 09:40:46,9,8,1934-11-02,ta,3591 Burton Junctions Suite 749,00517,Parkershire,SE,+1-666-343-0512x13500,2020-01-15 15:24:21,2020-01-15 15:24:21
9,2021-08-15 09:40:46,10,9,1947-09-14,tt,2744 Nicole Port Suite 898,42038,Velezborough,SO,4169241456,2020-01-15 16:04:34,2020-01-15 16:04:34


## List resulting files

In [8]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()]) + 2

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_path = '.' + os.sep + f 
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f_path:<{max_w}} | {f_size:>2d} KB")

21-08-15 09:40:46 | .\S3\staging\dms\abc\customers\customers\LOAD00000001.parquet        |  4 KB
21-08-15 09:40:46 | .\S3\staging\dms\abc\customers\customer_details\LOAD00000001.parquet |  9 KB
21-08-15 09:40:46 | .\S3\staging\dms\abc\devices\device_models\LOAD00000001.parquet      |  6 KB
21-08-15 09:40:46 | .\S3\staging\dms\abc\devices\devices\LOAD00000001.parquet            |  6 KB


## Preview only first lines

In [9]:
from IPython.display import HTML

panda_tables = [df_c, df_cd, df_dm, df_d]

for pt in panda_tables:
    display(HTML(pt.head(1).to_html(index=False)))

dms_timestamp,id,username,created,modified
2021-08-15 09:40:46,1,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56


dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
2021-08-15 09:40:46,1,1,1942-09-01,gez,19254 Le Shoal,96842,West Jessicafurt,BG,001-732-790-0908,1970-01-15 10:00:00,1970-01-15 10:00:00


dms_timestamp,id,release_date,name,color,description,created,modified
2021-08-15 09:40:46,1,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01


dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
2021-08-15 09:40:46,1,1,2,846-09-7018,1970-01-15 10:40:34,1970-01-15 11:19:42
